In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/data.json
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/outputs/cyan_square.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/outputs/blue_octagon.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/outputs/yellow_star.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/outputs/green_triangle.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/outputs/yellow_square.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/inputs/triangle.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/inputs/star.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/inputs/square.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/validation/inputs/octagon.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/training/data.json
/kaggle/input/ayna-ml-polygon-dataset/dataset/training/outputs/purple_octagon.png
/kaggle/input/ayna-ml-polygon-dataset/dataset/training/outputs/orange_squa

In [2]:
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    """(Convolution => [Batch Norm] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        # x1 is from the upsampling path, x2 is the skip connection from the downsampling path
        x1 = self.up(x1)
        
        # Concatenate the skip connection feature map with the upsampled feature map
        # The dimensions might not match perfectly if the input image size is not a power of 2
        # Therefore, we might need to pad x1 to match x2's dimensions
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = nn.functional.pad(x1, [diffX // 2, diffX - diffX // 2,
                                    diffY // 2, diffY - diffY // 2])
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    """Final output convolution layer"""
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class ConditionalUNet(nn.Module):
    def __init__(self, n_channels, n_classes, num_colors, embedding_dim=32, bilinear=True):
        """
        Conditional UNet from scratch.
        
        Args:
            n_channels (int): Number of channels in the input image (e.g., 1 for grayscale, 3 for RGB)
            n_classes (int): Number of channels in the output image (e.g., 3 for RGB color)
            num_colors (int): The number of possible colors (vocabulary size for embedding)
            embedding_dim (int): The dimension of the color embedding vector
            bilinear (bool): Whether to use bilinear upsampling or transposed convolutions
        """
        super(ConditionalUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        # --- Color Embedding ---
        # We will embed the color index into a vector of size `embedding_dim`
        self.color_embedding = nn.Embedding(num_colors, embedding_dim)

        # --- Encoder Path ---
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)

        # --- Bottleneck Conditioning ---
        # This linear layer will project the concatenated feature vector + color embedding 
        # to a dimension suitable for reshaping back into an image feature map.
        # Assuming an input image size of 128x128, the feature map at the bottleneck will be 8x8.
        # So, the flattened feature vector size is (1024 // factor) * 8 * 8.
        # This can be fragile if image size changes. A more robust way is to use adaptive pooling.
        # For now, let's make it flexible. We will calculate the size in forward pass.
        self.bottleneck_proj = nn.Linear((1024 // factor) + embedding_dim, 1024 // factor)
        
        # --- Decoder Path ---
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, image, color_idx):
        # 1. Encoder
        x1 = self.inc(image)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        # 2. Get Color Embedding
        color_emb = self.color_embedding(color_idx) # (batch_size, embedding_dim)
        
        # 3. Conditioning at the Bottleneck
        # Average pool the spatial dimensions of the bottleneck features
        pooled_features = torch.mean(x5, dim=[2, 3]) # (batch_size, num_features)
        
        # Concatenate pooled features with color embedding
        combined_vec = torch.cat([pooled_features, color_emb], dim=1)
        
        # Project the combined vector back to the original number of features
        projected_vec = self.bottleneck_proj(combined_vec)
        
        # Add this back to the original bottleneck features (as a residual)
        # We need to reshape the projected vector to match the spatial dimensions of x5
        # (batch_size, num_features) -> (batch_size, num_features, 1, 1)
        # This is then broadcast-added to x5
        conditioned_x5 = x5 + projected_vec.unsqueeze(-1).unsqueeze(-1)
        
        # 4. Decoder
        x = self.up1(conditioned_x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        
        # 5. Final Output
        logits = self.outc(x)
        
        # We can add a final activation if needed, e.g., Sigmoid for [0,1] or Tanh for [-1,1]
        # It's often better to leave this to the loss function (e.g., BCEWithLogitsLoss)
        # or apply it after training for inference. Let's use Sigmoid for this case.
        return torch.sigmoid(logits)

In [3]:
if __name__ == '__main__':
    # --- Configuration ---
    # These should match your dataset
    INPUT_CHANNELS = 1  # Grayscale input polygon
    OUTPUT_CHANNELS = 3 # RGB output colored polygon
    NUM_COLORS = 10     # Example: "red", "blue", "yellow", etc.
    IMG_SIZE = 128      # Example image size

    # --- Model Initialization ---
    model = ConditionalUNet(
        n_channels=INPUT_CHANNELS, 
        n_classes=OUTPUT_CHANNELS, 
        num_colors=NUM_COLORS
    )

    # --- Create Dummy Inputs ---
    # A batch of 4 grayscale images of size 128x128
    dummy_image_batch = torch.randn(4, INPUT_CHANNELS, IMG_SIZE, IMG_SIZE) 
    
    # A batch of 4 color indices. These should be integer IDs for your colors.
    # e.g., 0 for 'red', 1 for 'blue', etc.
    dummy_color_indices = torch.randint(0, NUM_COLORS, (4,)) # (batch_size,)

    # --- Forward Pass ---
    # Ensure model is in training mode if you plan to backpropagate
    model.train() 
    output_image_batch = model(dummy_image_batch, dummy_color_indices)

    # --- Check Output Shape ---
    print("Model instantiated successfully!")
    print(f"Input image batch shape: {dummy_image_batch.shape}")
    print(f"Input color indices shape: {dummy_color_indices.shape}")
    print(f"Output image batch shape: {output_image_batch.shape}")

    # The output shape should be (4, 3, 128, 128)
    assert output_image_batch.shape == (4, OUTPUT_CHANNELS, IMG_SIZE, IMG_SIZE)
    print("\nOutput shape is correct.")

Model instantiated successfully!
Input image batch shape: torch.Size([4, 1, 128, 128])
Input color indices shape: torch.Size([4])
Output image batch shape: torch.Size([4, 3, 128, 128])

Output shape is correct.


In [4]:
# ===================================================================
# STEP 0: IMPORT NECESSARY LIBRARIES
# ===================================================================
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

print("Libraries imported successfully.")

# ===================================================================
# STEP 1: DEFINE THE DATASET CLASS (FRESH IMPLEMENTATION)
# ===================================================================
import os
import json
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class PolygonDataset(Dataset):
    """
    Correctly loads data from a JSON file that is a list of records.
    """
    def __init__(self, data_json_path, root_dir, image_size=128):
        
        # Open and load the JSON file.
        with open(data_json_path, 'r') as f:
            # The diagnostic test proved that `json.load` returns a LIST directly.
            # So, we assign it directly to self.records.
            self.records = json.load(f)
            
        self.root_dir = root_dir
        
        # The rest of the code was correct and works on `self.records` as a list.
        all_colors = sorted(list(set(item['colour'] for item in self.records)))
        self.color_to_idx = {name: i for i, name in enumerate(all_colors)}
        
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]
        
        input_filename = record['input_polygon']
        output_filename = record['output_image']
        color_name = record['colour']
        
        input_path = os.path.join(self.root_dir, 'inputs', input_filename)
        output_path = os.path.join(self.root_dir, 'outputs', output_filename)
        
        input_image = Image.open(input_path).convert("L")
        output_image = Image.open(output_path).convert("RGB")
        
        input_tensor = self.transform(input_image)
        output_tensor = self.transform(output_image)
        
        color_index = self.color_to_idx[color_name]
        color_tensor = torch.tensor(color_index, dtype=torch.long)
        
        return input_tensor, color_tensor, output_tensor

    def get_num_colors(self):
        return len(self.color_to_idx)
# ===================================================================
# STEP 2: SET UP PATHS AND CONFIGURATIONS
# ===================================================================
# !! IMPORTANT: Replace 'ayna-ml-polygon-dataset' with your dataset name if different !!
BASE_INPUT_PATH = "/kaggle/input/ayna-ml-polygon-dataset/" 

# Check if the path exists to avoid FileNotFoundError
if not os.path.exists(BASE_INPUT_PATH):
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"!!! ERROR: The path '{BASE_INPUT_PATH}' does not exist.")
    print(f"!!! Please check the name of your Kaggle dataset.")
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
else:
    print(f"Base path '{BASE_INPUT_PATH}' found.\n")

    DATA_ROOT = os.path.join(BASE_INPUT_PATH, 'dataset')
    TRAIN_DIR = os.path.join(DATA_ROOT, 'training')
    TRAIN_JSON = os.path.join(TRAIN_DIR, 'data.json')

    IMAGE_SIZE = 128
    BATCH_SIZE = 8

    # ===================================================================
    # STEP 3: CREATE AND TEST THE DATASET AND DATALOADER
    # ===================================================================
    try:
        print(">>> Attempting to create the training dataset...")
        train_dataset = PolygonDataset(data_json_path=TRAIN_JSON, root_dir=TRAIN_DIR, image_size=IMAGE_SIZE)
        
        print(">>> Attempting to create the DataLoader...")
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        print("DataLoader created successfully.\n")
        
        print(">>> Attempting to fetch one batch of data...")
        input_batch, color_batch, output_batch = next(iter(train_loader))
        print("Successfully fetched one batch!\n")
        
        print("--- FINAL VERIFICATION ---")
        print(f"Input image batch shape:  {input_batch.shape}")
        print(f"Color index batch shape:  {color_batch.shape}")
        print(f"Output image batch shape: {output_batch.shape}")
        print("\n✅ SETUP COMPLETE AND VERIFIED! You can now proceed to training.")

    except Exception as e:
        print(f"\n❌ An error occurred during the process.")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Message: {e}")
        print("\nPlease review the diagnostic prints above to identify the issue.")

Libraries imported successfully.
Base path '/kaggle/input/ayna-ml-polygon-dataset/' found.

>>> Attempting to create the training dataset...
>>> Attempting to create the DataLoader...
DataLoader created successfully.

>>> Attempting to fetch one batch of data...
Successfully fetched one batch!

--- FINAL VERIFICATION ---
Input image batch shape:  torch.Size([8, 1, 128, 128])
Color index batch shape:  torch.Size([8])
Output image batch shape: torch.Size([8, 3, 128, 128])

✅ SETUP COMPLETE AND VERIFIED! You can now proceed to training.


In [5]:
# NEW AND CORRECT CODE FOR KAGGLE
!pip install wandb -q
import wandb
from kaggle_secrets import UserSecretsClient

# This line fetches the secret you just created
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# This line logs you in using the key
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: priyanshu-mohanty366 (priyanshu-mohanty366-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm # For progress bars
import wandb
import os

# ===================================================================
# NOTE: Ensure the model.py and dataset.py files are correctly defined
# in your environment, or that the classes are defined in cells above this one.
# For simplicity, let's assume the classes ConditionalUNet and PolygonDataset
# are already defined in the notebook's memory.
# ===================================================================

# --- 1. HYPERPARAMETER CONFIGURATION ---
# Using a dictionary makes it easy to log all hyperparameters to wandb
config = {
    "epochs": 50,          # Number of times to iterate over the full dataset
    "batch_size": 16,      # Number of samples per batch
    "learning_rate": 1e-4, # Step size for the optimizer
    "image_size": 128,     # The image resolution
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "dataset_name": "ayna-ml-polygon-dataset", # For Kaggle path
    "project_name": "ayna-ml-assignment-polygons" # Your wandb project name
}

print(f"Using device: {config['device']}")

# --- 2. SETUP WANDB ---
wandb.init(
    project=config['project_name'],
    config=config
)

# For saving the model artifact later
SAVE_DIR = "/kaggle/working/"
best_model_path = os.path.join(SAVE_DIR, "best_unet_model.pth")
best_val_loss = float('inf')


# --- 3. DATA LOADING ---
# Define paths based on Kaggle's file system
BASE_INPUT_PATH = f"/kaggle/input/{config['dataset_name']}/" 
DATA_ROOT = os.path.join(BASE_INPUT_PATH, 'dataset')
TRAIN_DIR = os.path.join(DATA_ROOT, 'training')
TRAIN_JSON = os.path.join(TRAIN_DIR, 'data.json')
VAL_DIR = os.path.join(DATA_ROOT, 'validation')
VAL_JSON = os.path.join(VAL_DIR, 'data.json')

# Create Datasets and DataLoaders
train_dataset = PolygonDataset(data_json_path=TRAIN_JSON, root_dir=TRAIN_DIR, image_size=config['image_size'])
val_dataset = PolygonDataset(data_json_path=VAL_JSON, root_dir=VAL_DIR, image_size=config['image_size'])

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=0, pin_memory=True)

print(f"Training on {len(train_dataset)} images, validating on {len(val_dataset)} images.")


# --- 4. MODEL, OPTIMIZER, LOSS FUNCTION ---
# Get the number of unique colors from the dataset itself
num_colors = train_dataset.get_num_colors()

# Initialize the model
model = ConditionalUNet(n_channels=1, n_classes=3, num_colors=num_colors).to(config['device'])

# L1Loss (Mean Absolute Error) is often better for image generation than MSE
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

# Tell wandb to watch the model's gradients
wandb.watch(model, criterion, log="all", log_freq=10)


# --- 5. TRAINING & VALIDATION LOOP ---
for epoch in range(config['epochs']):
    # --- Training Phase ---
    model.train()
    train_loss = 0.0
    
    # Use tqdm for a nice progress bar
    for input_img, color_idx, target_img in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Training]"):
        # Move data to the configured device (GPU)
        input_img = input_img.to(config['device'])
        color_idx = color_idx.to(config['device'])
        target_img = target_img.to(config['device'])

        # Forward pass
        predicted_img = model(input_img, color_idx)
        loss = criterion(predicted_img, target_img)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        wandb.log({"step_train_loss": loss.item()})

    avg_train_loss = train_loss / len(train_loader)

    # --- Validation Phase ---
    model.eval()
    val_loss = 0.0
    example_images = [] # For logging to wandb

    with torch.no_grad(): # Disable gradient calculations for validation
        for i, (input_img, color_idx, target_img) in enumerate(tqdm(val_loader, desc=f"Epoch {epoch+1}/{config['epochs']} [Validation]")):
            input_img = input_img.to(config['device'])
            color_idx = color_idx.to(config['device'])
            target_img = target_img.to(config['device'])

            predicted_img = model(input_img, color_idx)
            loss = criterion(predicted_img, target_img)
            val_loss += loss.item()
            
            # On the first batch of every validation run, log some images
            if i == 0:
                # Log up to 5 images from the batch
                num_images_to_log = min(config['batch_size'], 5)
                for j in range(num_images_to_log):
                    # Combine the input, ground truth, and prediction into one image for easy comparison
                    # We need to clone and move tensors to CPU for wandb logging
                    img_input = input_img[j].cpu()
                    img_target = target_img[j].cpu()
                    img_pred = predicted_img[j].cpu()
                    
                    example_images.append(wandb.Image(
                        img_input, caption=f"Input Polygon"
                    ))
                    example_images.append(wandb.Image(
                        img_target, caption=f"Ground Truth"
                    ))
                    example_images.append(wandb.Image(
                        img_pred, caption=f"Prediction (Epoch {epoch+1})"
                    ))

    avg_val_loss = val_loss / len(val_loader)
    
    # --- Logging for the epoch ---
    wandb.log({
        "epoch": epoch + 1,
        "avg_train_loss": avg_train_loss,
        "avg_val_loss": avg_val_loss,
        "validation_examples": example_images
    })
    
    print(f"Epoch {epoch+1}/{config['epochs']} -> Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
    # --- Save the best model ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"-> New best model saved to {best_model_path} with val loss: {best_val_loss:.4f}")

# --- 6. FINISH TRAINING ---
print("\nTraining finished!")
print(f"Best model saved at {best_model_path} with validation loss: {best_val_loss:.4f}")

# Mark the wandb run as finished
wandb.finish()

Using device: cuda


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250804_203306-wpi28i7d
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run zesty-feather-4
wandb: ⭐️ View project at https://wandb.ai/priyanshu-mohanty366-none/ayna-ml-assignment-polygons
wandb: 🚀 View run at https://wandb.ai/priyanshu-mohanty366-none/ayna-ml-assignment-polygons/runs/wpi28i7d


Training on 56 images, validating on 5 images.


Epoch 1/50 [Training]:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/50 [Validation]:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch 49/50 -> Train Loss: 0.2561, Val Loss: 0.2761
-> New best model saved to /kaggle/working/best_unet_model.pth with val loss: 0.2761


Epoch 50/50 [Training]:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 50/50 [Validation]:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 50/50 -> Train Loss: 0.2514, Val Loss: 0.2717
-> New best model saved to /kaggle/working/best_unet_model.pth with val loss: 0.2717

Training finished!
Best model saved at /kaggle/working/best_unet_model.pth with validation loss: 0.2717


wandb: uploading history steps 240-249, summary, console lines 89-96
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:  avg_train_loss █▇▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
wandb:    avg_val_loss ██▇▇▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁
wandb:           epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb: step_train_loss █▇▆▆▆▅▅▄▄▄▄▅▄▃▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:  avg_train_loss 0.25143
wandb:    avg_val_loss 0.27169
wandb:           epoch 50
wandb: step_train_loss 0.25637
wandb: 
wandb: 🚀 View run zesty-feather-4 at: https://wandb.ai/priyanshu-mohanty366-none/ayna-ml-assignment-polygons/runs/wpi28i7d
wandb: ⭐️ View project at: https://wandb.ai/priyanshu-mohanty366-none/ayna-ml-assignment-polygons
wandb: Synced 5 W&B file(s), 700 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250804_203306-wpi28i7d/logs
